### Carregar arquivo ###

In [1]:
CAMINHO_BANCO_DE_DADOS = '../../data/dados_analisados_2025-05-03.xlsx'

### Verificar pendências ###

In [3]:
from analise_corpus_todo.pendencias import identificar

pendencias = identificar(CAMINHO_BANCO_DE_DADOS)

Quantidade de pendências: 2170


#### Analise LLM ####

In [10]:
import pandas as pd
df_discursos = pd.read_excel(CAMINHO_BANCO_DE_DADOS, sheet_name='Discursos')


In [11]:
if 'CodigoPronunciamento' in df_discursos.columns:
    df_pendencias = df_discursos[df_discursos['CodigoPronunciamento'].isin(pendencias)]
else:
    raise ValueError("'CodigoPronunciamento' não encontrado em df_discursos.")

In [13]:
df_analise_corpus_todo = pd.read_excel(CAMINHO_BANCO_DE_DADOS, sheet_name='AnaliseCorpusTodo')

In [16]:
from analise_corpus_todo.analisar_com_llm import analisar_texto_arg_linguistica, analisar_texto_sumarizacao_classificacao
from analise_corpus_todo.prompts import instrucao_analise_arg_linguistica, instrucao_sumario_classificacao

#### Sumário

In [23]:
import json
import tempfile
import os
import pandas as pd
import numpy as np

# Substituir os NaN em df_pendencias antes da iteração
df_pendencias = df_pendencias.fillna("")  # Substituir NaN por espaço ("")
print("Substituídos todos os NaN por espaço no DataFrame `df_pendencias`.")

# Definir o caminho do arquivo temporário
temp_file_path = 'resultados_parciais.json'

# Inicializar dicionário para armazenamento dos resultados
if os.path.exists(temp_file_path):
    print(f"Arquivo parcial encontrado: {temp_file_path}. Carregando progresso anterior...")
    with open(temp_file_path, 'r', encoding='utf-8') as temp_file:
        try:
            resultados = json.load(temp_file)
            print("Progresso carregado com sucesso!")
        except json.JSONDecodeError:
            print(f"Erro ao carregar o arquivo JSON parcial {temp_file_path}. Inicializando novo progresso.")
            resultados = {}
else:
    print("Nenhum arquivo parcial encontrado. Iniciando novo processamento...")
    resultados = {}


# Função auxiliar para normalizar valores inválidos no JSON
def json_safe(data):
    if isinstance(data, dict):
        return {k: json_safe(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [json_safe(x) for x in data]
    elif isinstance(data, float):
        # Normalizar valores float inválidos (NaN, Inf, -Inf)
        if data != data or data == float("inf") or data == float("-inf"):
            return None
        return data
    else:
        return data


# Iterar sobre cada linha do DataFrame com print statements e salvar progresso
salvar_a_cada = 5  # Número de iterações antes de salvar
contador = 0  # Contador de progresso

total_pendencias = len(df_pendencias)
print(f"Iniciando o processamento de {total_pendencias} pendências...")

for index, linha in df_pendencias[['CodigoPronunciamento', 'TextoIntegral']].iterrows():
    codigo = linha['CodigoPronunciamento']
    texto = linha['TextoIntegral']

    # Checar se já foi processado
    if codigo in resultados:
        print(f"[{index}] Código {codigo} já processado. Pulando para o próximo...")
        continue

    print(f"[{index}] Processando código {codigo}...")

    # Aplicar a função analisar_texto_sumarizacao_classificacao
    try:
        analise = analisar_texto_sumarizacao_classificacao(codigo, instrucao_sumario_classificacao, texto)
    except Exception as e:
        print(f"Erro ao processar o código {codigo}: {e}")
        analise = "{}"  # Valor padrão em caso de falha na função

    # Tentar converter a string JSON em um dicionário
    try:
        analise_dict = json.loads(analise)  # Fazer parsing do JSON para dicionário
    except json.JSONDecodeError:
        print(f"Erro ao parsear JSON para o código {codigo}. Ignorando e continuando...")
        analise_dict = {}

    # Normalizar valores inválidos no JSON
    analise_dict = json_safe(analise_dict)

    # Selecionar apenas as chaves desejadas para salvar no JSON
    analise_filtrada = {
        'SentimentoGeral': analise_dict.get('SentimentoGeral'),
        'SentimentoConstituicao': analise_dict.get('SentimentoConstituicao'),
        'SumarioConstituicao': analise_dict.get('SumarioConstituicao'),
        'TrechosConstituicao': analise_dict.get('TrechosConstituicao'),
        'NovaConstituinteOuConstituicao_resposta': analise_dict.get('NovaConstituinteOuConstituicao', {}).get(
            'resposta'),
        'NovaConstituinteOuConstituicao_trecho': analise_dict.get('NovaConstituinteOuConstituicao', {}).get('trecho')
    }

    # Guardar o dicionário no resultado final
    resultados[codigo] = analise_filtrada
    print(f"[{index}] Código {codigo} processado com sucesso.")

    # Incrementar o contador
    contador += 1

    # Salvar os resultados parcialmente a cada 5 iterações
    if contador % salvar_a_cada == 0:
        print(f"Salvando progresso após {contador} iterações no arquivo {temp_file_path}...")
        with open(temp_file_path, 'w', encoding='utf-8') as temp_file:
            json.dump(resultados, temp_file, ensure_ascii=False, indent=4)
        print(f"Progresso salvo com sucesso!")

# Salvar quaisquer dados restantes após concluir o loop
print(f"Salvando o progresso final no arquivo {temp_file_path}...")
with open(temp_file_path, 'w', encoding='utf-8') as temp_file:
    json.dump(resultados, temp_file, ensure_ascii=False, indent=4)

print("Processamento concluído. Resultados salvos com sucesso!")


Substituídos todos os NaN por espaço no DataFrame `df_pendencias`.
Nenhum arquivo parcial encontrado. Iniciando novo processamento...
Iniciando o processamento de 2170 pendências...
[67349] Processando código 512328...
[67349] Código 512328 processado com sucesso.
[67350] Processando código 511720...
[67350] Código 511720 processado com sucesso.
[67351] Processando código 511641...
[67351] Código 511641 processado com sucesso.
[67352] Processando código 511622...
[67352] Código 511622 processado com sucesso.
[67353] Processando código 511550...
[67353] Código 511550 processado com sucesso.
Salvando progresso após 5 iterações no arquivo resultados_parciais.json...
Progresso salvo com sucesso!
[67354] Processando código 511539...
[67354] Código 511539 processado com sucesso.
[67355] Processando código 511529...
[67355] Código 511529 processado com sucesso.
[67356] Processando código 511505...
[67356] Código 511505 processado com sucesso.
[67357] Processando código 511490...
[67357] Códig

##### Análise linguística

In [27]:
import json
import tempfile
import os
import shutil
import pandas as pd

# Definir o caminho do arquivo de resultados parciais para `analisar_texto_arg_linguistica`
resultados_json = 'resultados_arg_linguistica.json'

# Inicializar ou carregar progresso anterior
if os.path.exists(resultados_json):
    print(f"Arquivo parcial encontrado: {resultados_json}. Carregando progresso anterior...")
    with open(resultados_json, 'r', encoding='utf-8') as json_file:
        try:
            resultados = json.load(json_file)
            print("Progresso carregado com sucesso!")
        except json.JSONDecodeError:
            print(f"Erro ao carregar o arquivo parcial {resultados_json}. Inicializando novo progresso.")
            resultados = {}
else:
    print("Nenhum arquivo parcial encontrado. Iniciando novo processamento...")
    resultados = {}


# Função auxiliar para normalizar valores inválidos no JSON
def json_safe(data):
    if isinstance(data, dict):
        return {k: json_safe(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [json_safe(x) for x in data]
    elif isinstance(data, float):
        # Normalizar valores float inválidos (NaN, Inf, -Inf)
        if data != data or data == float("inf") or data == float("-inf"):
            return None
        return data
    else:
        return data


# Iterar sobre cada linha do DataFrame com print statements e salvar progresso incrementalmente
salvar_a_cada = 5  # Número de iterações antes de salvar
contador = 0  # Contador de progresso
total_pendencias = len(df_pendencias)
print(f"Iniciando o processamento de {total_pendencias} pendências com `analisar_texto_arg_linguistica`...")

for index, linha in df_pendencias[['CodigoPronunciamento', 'TextoIntegral']].iterrows():
    codigo = linha['CodigoPronunciamento']
    texto = linha['TextoIntegral']

    # Checar se já foi processado
    if codigo in resultados:
        print(f"[{index}] Código {codigo} já processado. Pulando para o próximo...")
        continue

    print(f"[{index}] Processando código {codigo} com `analisar_texto_arg_linguistica`...")

    # Aplicar a função analisar_texto_arg_linguistica
    try:
        analise = analisar_texto_arg_linguistica(codigo, instrucao_analise_arg_linguistica, texto)
    except Exception as e:
        print(f"Erro ao processar o código {codigo}: {e}")
        analise = "{}"  # Valor padrão em caso de falha na função

    # Tentar converter a string JSON em um dicionário
    try:
        analise_dict = json.loads(analise)  # Fazer parsing do JSON para dicionário
    except json.JSONDecodeError:
        print(f"Erro ao parsear JSON para o código {codigo}. Ignorando e continuando...")
        analise_dict = {}

    # Normalizar valores inválidos no JSON
    analise_dict = json_safe(analise_dict)

    # Adicionar os resultados normalizados ao dicionário final
    resultados[codigo] = analise_dict
    print(f"[{index}] Código {codigo} processado com sucesso.")

    # Incrementar o contador e salvar os resultados parcialmente
    contador += 1
    if contador % salvar_a_cada == 0:
        print(f"Salvando progresso após {contador} iterações no arquivo {resultados_json}...")
        with open(resultados_json, 'w', encoding='utf-8') as json_file:
            json.dump(resultados, json_file, ensure_ascii=False, indent=4)
        print(f"Progresso salvo com sucesso!")

# Salvar quaisquer dados restantes após concluir o loop
print(f"Salvando o progresso final no arquivo {resultados_json}...")
with open(resultados_json, 'w', encoding='utf-8') as json_file:
    json.dump(resultados, json_file, ensure_ascii=False, indent=4)
print("Processamento concluído. Resultados salvos com sucesso!")


Nenhum arquivo parcial encontrado. Iniciando novo processamento...
Iniciando o processamento de 2170 pendências com `analisar_texto_arg_linguistica`...
[67349] Processando código 512328 com `analisar_texto_arg_linguistica`...
[67349] Código 512328 processado com sucesso.
[67350] Processando código 511720 com `analisar_texto_arg_linguistica`...
[67350] Código 511720 processado com sucesso.
[67351] Processando código 511641 com `analisar_texto_arg_linguistica`...
[67351] Código 511641 processado com sucesso.
[67352] Processando código 511622 com `analisar_texto_arg_linguistica`...
[67352] Código 511622 processado com sucesso.
[67353] Processando código 511550 com `analisar_texto_arg_linguistica`...
[67353] Código 511550 processado com sucesso.
Salvando progresso após 5 iterações no arquivo resultados_arg_linguistica.json...
Progresso salvo com sucesso!
[67354] Processando código 511539 com `analisar_texto_arg_linguistica`...
[67354] Código 511539 processado com sucesso.
[67355] Processan

#### Atualizando xlsx

In [65]:
import pandas as pd
import json

# Insira os caminhos dos seus arquivos locais
xlsx_path = '../../data/dados_analisados_2025-05-03.xlsx'
json_path_1 = 'resultados_parciais.json'
json_path_2 = 'resultados_arg_linguistica.json'
output_path = '../../data/dados_analisados_2025-05-03_2.xlsx'

# Carregar todas as planilhas originais
excel_data = pd.ExcelFile(xlsx_path)
sheet_names = excel_data.sheet_names

# Carregar a planilha AnaliseCorpusTodo para modificação
df_analise = excel_data.parse('AnaliseCorpusTodo')

# Carregar e ajustar JSON resultados_parciais.json
with open(json_path_1, 'r', encoding='utf-8') as f:
    json_data_1 = json.load(f)
df_json_1 = pd.DataFrame.from_dict(json_data_1, orient='index')
df_json_1.index = df_json_1.index.astype(df_analise.index.dtype)

# Carregar e ajustar JSON resultados_arg_linguistica.json
with open(json_path_2, 'r', encoding='utf-8') as f:
    json_data_2 = json.load(f)
df_json_2 = pd.DataFrame.from_dict(json_data_2, orient='index')
df_json_2.index = df_json_2.index.astype(df_analise.index.dtype)

# Combinar os dados originais com os JSONs, garantindo inclusão de novas linhas
df_analise = df_json_1.combine_first(df_analise)
df_analise = df_json_2.combine_first(df_analise)
df_analise.reset_index(inplace=True)

def normalize_dataframe(df):
    """
    Normaliza um DataFrame removendo caracteres inválidos de strings.

    Parâmetros:
    - df (pd.DataFrame): DataFrame a ser normalizado.

    Retorna:
    - pd.DataFrame: Um novo DataFrame com os valores normalizados.
    """

    def clean_value(value):
        # Se for string, remove caracteres não imprimíveis (abaixo de 32) e substitui espaços extras.
        if isinstance(value, str):
            # Remove caracteres ASCII não imprimíveis
            value = re.sub(r"[\x00-\x1F]+", "", value)
            # Remove espaços extras desnecessários
            return value.strip()
        return value

    # Aplica a função de limpeza célula a célula no DataFrame
    return df.applymap(clean_value)



# Criar novo arquivo Excel com as planilhas preservadas e a modificada
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for sheet in sheet_names:
        if sheet != 'AnaliseCorpusTodo':
            temp_df = excel_data.parse(sheet)
            temp_df.to_excel(writer, sheet_name=sheet, index=False)

    # Normalizar o DataFrame
    df_analise_normalizado = normalize_dataframe(df_analise)

    # Adicionar a planilha modificada
    df_analise_normalizado.to_excel(writer, sheet_name='AnaliseCorpusTodo', index=False)

print(f'Novo arquivo Excel criado com sucesso: {output_path}')


/var/folders/2k/hh_m8b1d5mn22m0sk7d_zncc0000gn/T/ipykernel_28115/4209742206.py:55: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(clean_value)


Novo arquivo Excel criado com sucesso: ../../data/dados_analisados_2025-05-03_2.xlsx


Verificando integridade do banco de dados

In [66]:
import pandas as pd

# Caminho do arquivo Excel
caminho_arquivo = '../../data/dados_analisados_2025-05-03_2.xlsx'

# Carregar planilhas
df_discursos = pd.read_excel(caminho_arquivo, sheet_name='Discursos')
df_analise_corpus_todo = pd.read_excel(caminho_arquivo, sheet_name='AnaliseCorpusTodo')

# Verificar se as colunas relevantes estão presentes
if 'CodigoPronunciamento' not in df_discursos.columns:
    raise KeyError("A coluna 'CodigoPronunciamento' não está presente na planilha 'Discursos'.")
if 'CodigoPronunciamento' not in df_analise_corpus_todo.columns:
    raise KeyError("A coluna 'CodigoPronunciamento' não está presente na planilha 'AnaliseCorpusTodo'.")

# Verificar se todos os discursos estão presentes
faltantes = df_discursos[~df_discursos['CodigoPronunciamento'].isin(df_analise_corpus_todo['CodigoPronunciamento'])]

# Exibir resultados da verificação
if faltantes.empty:
    print("Todos os discursos da planilha 'Discursos' estão presentes na planilha 'AnaliseCorpusTodo'.")
else:
    print(f"Existem {len(faltantes)} discursos ausentes na planilha 'AnaliseCorpusTodo'.")
    print("Esses discursos ausentes são:")
    print(faltantes)

# Remover o índice da planilha 'AnaliseCorpusTodo'
df_analise_corpus_todo.reset_index(inplace=True, drop=True)

Todos os discursos da planilha 'Discursos' estão presentes na planilha 'AnaliseCorpusTodo'.


##### Para facilitar o trabalho de análise, resolvi deixar para trabalhar com XLSX quando for criar gráficos no Tableau! #####

In [85]:
import os
import pandas as pd
import csv  # Biblioteca para controle avançado sobre o formato CSV

# Caminho para o arquivo Excel e o diretório onde serão salvos os CSVs
caminho_arquivo_excel = '../../data/dados_analisados_2025-05-03_2.xlsx'
diretorio_saida = '../../data/temp'

# Criar o diretório de saída, caso ainda não exista
if not os.path.exists(diretorio_saida):
    os.makedirs(diretorio_saida)
    print(f"Diretório '{diretorio_saida}' criado com sucesso.")

# Abrir o arquivo Excel e salvar cada planilha como CSV
excel_data = pd.ExcelFile(caminho_arquivo_excel)  # Carregar o arquivo Excel
for sheet_name in excel_data.sheet_names:
    df = excel_data.parse(sheet_name)  # Carregar a planilha como DataFrame
    caminho_csv = os.path.join(diretorio_saida, f"{sheet_name}.csv")  # Definir o caminho do CSV
    df.to_csv(
        caminho_csv,
        index=False,  # Não salvar índice do DataFrame
        encoding='utf-8',  # Garantir que a codificação seja UTF-8
        sep=',',  # Separador padrão: vírgula
        quoting=csv.QUOTE_ALL,  # Envolver todas as células em aspas
    )
    print(f"Planilha '{sheet_name}' salva como CSV em: {caminho_csv}")

print("Todas as planilhas foram convertidas para arquivos CSV em 'data/temp'.")


Planilha 'Discursos' salva como CSV em: ../../data/temp/Discursos.csv
Planilha 'Senadores' salva como CSV em: ../../data/temp/Senadores.csv
Planilha 'PesquisaSemantica' salva como CSV em: ../../data/temp/PesquisaSemantica.csv
Planilha 'AnaliseAnual' salva como CSV em: ../../data/temp/AnaliseAnual.csv
Planilha 'TopicosCorpusTodo' salva como CSV em: ../../data/temp/TopicosCorpusTodo.csv
Planilha 'AnaliseCorpusTodo' salva como CSV em: ../../data/temp/AnaliseCorpusTodo.csv
Todas as planilhas foram convertidas para arquivos CSV em 'data/temp'.


##### Limpeza de textos ####

In [86]:
import pandas as pd
import re

# Caminho para o arquivo CSV
caminho_csv_discursos = '../../data/temp/Discursos.csv'

# Carregar o arquivo CSV
df_discursos = pd.read_csv(caminho_csv_discursos, encoding='utf-8')

# Lista de padrões e substituições
substituicoes = {
    r"\bComissão de Constituição, Justiça e Cidadania\b": "CCJC",
    r"\bComissão de Constituição e Justiça\b": "CCJ",
    r"\bProposta de Emenda à Constituição\b": "PEC",
    r"\bPropostas de Emenda à Constituição\b": "PEC",
}


# Função para remover acentos e aplicar substituições
def substituir_expressao(texto, substituicoes):


    # Substituir expressões conforme os padrões listados
    for padrao, substituto in substituicoes.items():
        texto = re.sub(padrao, substituto, texto, flags=re.IGNORECASE)
    return texto


# Aplicar a substituição em todas as células do DataFrame
df_discursos = df_discursos.applymap(
    lambda x: substituir_expressao(x, substituicoes) if isinstance(x, str) else x
)

# Salvar o arquivo atualizado
df_discursos.to_csv(caminho_csv_discursos, index=False, encoding='utf-8', quoting=1)

print(f"As expressões foram substituídas e o arquivo atualizado foi salvo em: {caminho_csv_discursos}")


/var/folders/2k/hh_m8b1d5mn22m0sk7d_zncc0000gn/T/ipykernel_28115/1216596194.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_discursos = df_discursos.applymap(


As expressões foram substituídas e o arquivo atualizado foi salvo em: ../../data/temp/Discursos.csv


#### TF-IDF e BM25 Corpus Todo ####

In [72]:
!pip install scikit-learn rank-bm25

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/25/92/ee1d7a00bb6b8c55755d4984fd82608603a3cc59959245068ce32e7fb808/scikit_learn-1.6.1-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Using cached scikit_learn-1.6.1-cp311-cp311-macosx_12_0_arm64.whl.metadata (31 kB)
  Obtaining dependency information for rank-bm25 from https://files.pythonhosted.org/packages/2a/21/f691fb2613100a62b3fa91e9988c991e9ca5b89ea31c0d3152a3210344f9/rank_bm25-0.2.2-py3-none-any.whl.metadata
  Using cached rank_bm25-0.2.2-py3-none-any.whl.metadata (3.2 kB)
  Obtaining dependency information for scipy>=1.6.0 from https://files.pythonhosted.org/packages/61/d8/84da3fffefb6c7d5a16968fe5b9f24c98606b165bb801bb0b8bc3985200f/scipy-1.15.2-cp311-cp311-macosx_14_0_arm64.whl.metadata
  Using cached scipy-1.15.2-cp311-cp311-macosx_14_0_arm64.whl.metadata (61 kB)
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/da/d3/13ee227a148af1c69

In [87]:
import pandas as pd
import numpy as np
from rank_bm25 import BM25Okapi
import re

# Caminhos dos arquivos
caminho_csv_discursos = '../../data/temp/Discursos.csv'  # Arquivo de entrada
caminho_csv_saida = '../../data/temp/AnaliseCorpusTodo.csv'  # Arquivo de saída (já existente)

# Carregar o arquivo de saída existente
df_saida = pd.read_csv(caminho_csv_saida, encoding='utf-8')

# Verificar a presença da coluna "BM25_CorpusTodo_Constituicao"
if 'BM25_CorpusTodo_Constituição' not in df_saida.columns:
    raise KeyError("A coluna 'BM25_CorpusTodo_Constituicao' não foi encontrada em AnaliseCorpusTodo.csv.")

# Carregar o arquivo de entrada (discursos) e pré-processar os textos
df_discursos = pd.read_csv(caminho_csv_discursos, encoding='utf-8')

# Verificar a presença da coluna "TextoIntegral"
if 'TextoIntegral' not in df_discursos.columns:
    raise KeyError("A coluna 'TextoIntegral' não foi encontrada no arquivo Discursos.csv.")

# Pré-processamento e remoção de acentos, preservando letras maiúsculas
from unidecode import unidecode  # Certifique-se de instalar a biblioteca unidecode


def preprocessar_texto(texto):
    texto = re.sub(r'[^\w\s]', '', texto, flags=re.UNICODE)  # Remove pontuações
    texto = unidecode(texto)  # Remove acentos, mantendo maiúsculas
    return texto


# Preenchendo valores NaN e aplicando pré-processamento
df_discursos['TextoIntegral'] = df_discursos['TextoIntegral'].fillna('')
df_discursos['Texto_Tratado'] = df_discursos['TextoIntegral'].apply(preprocessar_texto)

# Calcular BM25 para a palavra "constituição"
palavra_alvo = 'Constituicao'  # Sem acento, pré-processada

# Preparar documentos tokenizados
documentos_tokenizados = [doc.split() for doc in df_discursos['Texto_Tratado']]
bm25 = BM25Okapi(documentos_tokenizados)
bm25_scores = bm25.get_scores([palavra_alvo])  # Obter os escores para a palavra

# Atualizar a coluna "BM25_CorpusTodo_Constituicao" no DataFrame do arquivo de saída
if len(df_saida) != len(bm25_scores):
    raise ValueError("Os tamanhos dos DataFrames 'Discursos.csv' e 'AnaliseCorpusTodo.csv' não coincidem.")

df_saida['BM25_CorpusTodo_Constituição'] = bm25_scores

# Salvar as alterações no arquivo de saída, preservando as demais colunas
df_saida.to_csv(caminho_csv_saida, index=False, encoding='utf-8', quoting=1)

print(f"A coluna 'BM25_CorpusTodo_Constituicao' foi atualizada com sucesso em: {caminho_csv_saida}")


A coluna 'BM25_CorpusTodo_Constituicao' foi atualizada com sucesso em: ../../data/temp/AnaliseCorpusTodo.csv


#### Análise Anual ####